In [0]:
# input files: whole dataset, paragraphs file
# ouput: Zip file containing testing dataset with selected paragraph

In [0]:
import numpy as np
from gensim.models import Word2Vec
import re
import json
import glob
import os
from sklearn.metrics.pairwise import cosine_similarity
import zipfile

In [0]:
basePath = '/content/'

data_dir = basePath+'dataset'
drive_dir = basePath+'drive/My Drive/temp/DatasetN'
paraFile = basePath + 'paragraph2.json'

'''data_dir = basePath+'tempdataset'
drive_dir = basePath+'drive/My Drive/temp/tempDataset'
paraFile = basePath + 'temp.json' '''

outDir = basePath+'test'
alpha = 0.001

In [0]:
'''for file_name in os.listdir(drive_dir):
    if file_name.endswith('.zip'):
        with zipfile.ZipFile(drive_dir+'/'+file_name,'r') as zip_dir:
            zip_dir.extractall(path='/content/')'''

"for file_name in os.listdir(drive_dir):\n    if file_name.endswith('.zip'):\n        with zipfile.ZipFile(drive_dir+'/'+file_name,'r') as zip_dir:\n            zip_dir.extractall(path='/content/')"

In [0]:
def sifEmbeddings(sentences, model, alpha):

    vocab = model.wv.vocab
    embs = model.wv       
    size = model.vector_size  
    
    total = 0
    for word in vocab:
        total += vocab[word].count 

    output = []
    
    for sent in sentences:
        count = 0
        v = np.zeros(size, dtype=np.float32) 
        for word in sent:
          if word in vocab:
            #print(word)
            weight = alpha / (alpha + (vocab[word].count/total))
            v += weight * embs[word]
            count += 1 
                
        if count > 0:
            v *= 1/count

        output.append(v)
    return np.vstack(output)

In [0]:
def splitSent(sent):
    regX = '[\s()\.\?]'
    splitSent = re.split(regX,sent)
    splitSent = list(filter(None,splitSent))
    return splitSent

In [0]:
def getFileSent(inpFile):
    fileSent = ''
    article = ''
    with open(inpFile,'r') as f:
        x = json.loads(f.read())
        article = x['article']
        for option in x['options'][0]:
            fileSent += option
            fileSent += ' '
        fileSent += x['questions'][0]
    return fileSent,article

In [0]:
def getDatasetAsSents():
    fileSents = []
    dirs = os.listdir(data_dir)
    print(dirs)
    for d in dirs:
      filenames = glob.glob(os.path.join(data_dir,d)+"/*json")
      for file in filenames:
          with open(file,'r') as f:
              x = json.loads(f.read())
              for option in x['options'][0]:
                  fileSents.append(option)
              fileSents.append(x['questions'][0])
              fileSents.append(x['article'])
    return fileSents

In [0]:
def getQuesEmb(model,inpFile):
    fileSent,article = getFileSent(inpFile)
    fileSent = fileSent + (' ' + fileSent)*5
    sent = splitSent(fileSent)
    emb = sifEmbeddings([sent], model, alpha=alpha)[0]
    return emb,article

In [0]:
'''def getMatchingPara(quesEmb,paraEmbs):
    #paraEmbs = np.loadtxt('paraEmbs.csv',delimiter=',')

    scores = []
    for paraEmb in paraEmbs:
        score=float( cosine_similarity([paraEmb],[quesEmb]) )
        scores.append(score)
    #print(scores)
    mxScoreInd = scores.index(max(scores))
    mxScoreInd += 1

    with open(paraFile,'r') as f:
      x = json.loads(f.read())
      return x['para'+str(mxScoreInd)]'''

"def getMatchingPara(quesEmb,paraEmbs):\n    #paraEmbs = np.loadtxt('paraEmbs.csv',delimiter=',')\n\n    scores = []\n    for paraEmb in paraEmbs:\n        score=float( cosine_similarity([paraEmb],[quesEmb]) )\n        scores.append(score)\n    #print(scores)\n    mxScoreInd = scores.index(max(scores))\n    mxScoreInd += 1\n\n    with open(paraFile,'r') as f:\n      x = json.loads(f.read())\n      return x['para'+str(mxScoreInd)]"

In [0]:
def getMatchingPara(quesEmb,paraEmbs):
    #paraEmbs = np.loadtxt('paraEmbs.csv',delimiter=',')

    scores = []
    for paraEmb in paraEmbs:
        score=float( cosine_similarity([paraEmb],[quesEmb]) )
        scores.append(score)
    #print(scores)
    #mxScoreInd = scores.index(max(scores))
    #mxScoreInd += 1
    topK = 3
    #a = [2,6,5,9,7,2,3,6]
    topIds = np.argsort(scores)[-topK:]
    topIds += 1
    #topVal = [scores[i] for i in topIds]

    with open(paraFile,'r') as f:
      x = json.loads(f.read())
      for id in topIds:
          print(x['para'+str(id)])
      return x['para'+str(topIds[-1])]

In [0]:
def saveFile(inpFile,outFile,para):
    with open(inpFile,'r') as f:
      x = json.loads(f.read())
      x['article'] = para

      with open(outFile, 'w') as jsonOut:
          json.dump(x, jsonOut)

# **Word2Vec**

In [0]:
fileSents = getDatasetAsSents()
sentences = [ splitSent(sent) for sent in fileSents]

['dev', 'test', 'train']


In [0]:
model = Word2Vec(sentences, min_count=1, size=200)
#model.save(basePath+"word2vec.model")

# **Embedding Generation**

In [0]:
f = open(paraFile,'r')
x = json.loads(f.read())
f.close()
fileSents = [ x['para'+str(id)] for id in range(1,len(x)+1)]
sentences = [ splitSent(sent) for sent in fileSents]

In [0]:
#model = Word2Vec.load("word2vec.model")
paraEembs = sifEmbeddings(sentences, model, alpha=alpha)
print(paraEembs.shape)
#np.savetxt(basePath+'paraEmbs.csv',paraEembs, delimiter=',',fmt='%.15f')

(12304, 200)


# **Generate Testing Dataset**

In [0]:
filenames = glob.glob(os.path.join(data_dir,'test')+"/*json")
if not os.path.exists(outDir):
    os.mkdir(outDir)
count = 0
ind = 0
for filename in filenames:
    if ind == 10:
      break
    emb,article = getQuesEmb(model,filename)
    print()
    para = getMatchingPara(emb,paraEembs)
    #print()
    print(article)
    #print(para)
    if article == para:
        count += 1
    name = filename.split('/')[-1]
    #print(name)
    saveFile(filename,os.path.join(outDir,name),para)
    ind += 1

print()
print('accuracy: ',(count/len(filenames))*100)


The first stage of cellular respiration is glycolysis . It takes place in the cytosol of the cytoplasm.
The Calvin cycle is part of the light-independent reactions of photosynthesis. The Calvin cycle uses ATP and NADPH. The Krebs cycle is part of cellular respiration. This cycle makes ATP and NAPH.
The Krebs cycle is the second stage of cellular respiration.
The first stage of cellular respiration is glycolysis . It does not require oxygen, and it does not take place in the mitochondrion - it takes place in the cytosol of the cytoplasm.

The axe head and the doorstop are both examples of a wedge, a type of simple machine.
Base units have independent scales and cannot be described by a combination of any other base units. Examples of base units used in chemistry are length, mass, temperature, and time.
Derived units can be expressed as some combination of base units. Examples of derived units are area, volume, and speed.
Derived units can be expressed as some combination of base units.

In [0]:
with zipfile.ZipFile(basePath+'testdata.zip','w') as zf:
    for dirname, subdirs, files in os.walk(outDir):
        for filename in files:
            zf.write(os.path.join(dirname.split('/')[-1], filename))

In [0]:
#print('temp')

In [0]:
'''import numpy as np

topK = 3
a = [2,6,5,9,7,2,3,6]
topIds = np.argsort(a)[-topK:]
topVal = [a[i] for i in topIds]
print(topIds)
print(topVal)'''

[7 4 3]
[6, 7, 9]
